In [295]:
import pandas as pd
import numpy as np
import email
import os
from email.parser import Parser
import re

In [296]:
ham_dir = '/media/sf_Windows/SPAM_ASSASSIN/HAM/'
ham_emails = os.listdir(ham_dir)
spam_dir = '/media/sf_Windows/SPAM_ASSASSIN/SPAM/'
spam_emails = os.listdir(spam_dir)

In [278]:
emails = os.listdir(ham_dir+ham_emails[0]+'/')

In [297]:
import email

In [279]:
ham_dir+ham_emails[0]+'/'+emails[0]

'/media/sf_Windows/SPAM_ASSASSIN/HAM/easy_ham_2002/2475.8572d77868481d0fe409eedd8fa6d443'

In [280]:
f = open(ham_dir+ham_emails[0]+'/'+emails[0], 'r')
msg = f.read()
f.close()

In [229]:
content = results.get_payload()

In [233]:
content[2].get_payload()

[<email.message.Message instance at 0x7f0afdb7a3f8>]

In [216]:
def processContent(content):
    if type(content) != str:
        message = [subContent.get_payload().split('\n') for subContent in content]
        return ' '.join(reduce(lambda x, y: x + y, message, []))
    else: 
        return ' '.join(content.split('\n'))

In [281]:
from email.parser import Parser

In [282]:
results = Parser().parsestr(msg)

In [261]:
results.get_payload()[2].get_payload()[0].get_payload().get_payload

IndexError: list index out of range

In [154]:
len(b)

2

In [284]:
' '.join([i for i in walkMsg(results)]).

'URL: http://www.newsisfree.com/click/-2,8653742,1440/\nDate: Not supplied\n\nThe latest "Red List" adds 124 to the 11,000 endangered species around the \nglobe - but also includes a stick insect revival\n\n\n'

In [316]:
# extract the body of email
def walkMsg(msg):
    """
    Input: string of email
    Output: body of email
    """
    for part in msg.walk():
        if part.get_content_type() == "multipart/alternative":
            continue
        yield part.get_payload(decode=1)
# extract the information of email
# still need some works to get number of cc and bcc
def info_of_email(path,List_of_files,classification):
    """
    Input: path, list of files name, classification(spam or ham)
    Ourput: a list with info of email
    """
    parser = Parser()
    info = []
    for files in List_of_files:
        emailText =  open(path + files, 'rb').read()
        email = parser.parsestr(emailText)
        if email.get('cc') == None:
            no_cc = 0
        else:
            no_cc = len(email.get('cc'))
        if email.get("bcc") == None:
            no_bcc = 0
        else:
            no_bcc = len(email.get('bcc'))
        Date = email.get("Date")
        special_chars = re.compile(r"\t\n\r")
        try:
            Date = re.split(':|,| ',Date)
            info.append([Date[4], Date[3], Date[2], Date[0], Date[5], email.get("From"), 
                         email.get('To'), no_cc, no_bcc, 
                         str(email.get("Content-Type")).split(";")[0],
                         email.get('subject'), 
                         " ".join([i for i in walkMsg(email) if 
                                   i != None]).replace("\n"," ").replace("\t"," "),
                         classification])
        except:
            info.append([[], [], [], [], [], email.get("From"), 
                         email.get('To'), no_cc, no_bcc, 
                         str(email.get("Content-Type")).split(";")[0],
                         email.get('subject'), 
                         " ".join([i for i in walkMsg(email) if 
                                   i != None]).replace("\n"," ").replace("\t"," "),
                         classification])
    return info

In [317]:
ham_dir+ham_folder+'/'

'/media/sf_Windows/SPAM_ASSASSIN/HAM/easy_ham_2003/'

In [318]:
ham_results = []
for ham_folder in ham_emails:
    emails = os.listdir(ham_dir+ham_folder)
    results.append(info_of_email(ham_dir+ham_folder+'/',emails,'ham'))

In [319]:
hamList = [pd.DataFrame(df,
                      columns=['Year','Month','Day','Weekday',"Hour",
                                'From','To','cc','bcc','Content_type','Subjects','Content','Classification'])for df in results]
hamDF = pd.concat(hamList,ignore_index=True)

In [334]:
spam_results = []
for spam_folder in spam_emails:
    emails = os.listdir(spam_dir+spam_folder)
    spam_results.append(info_of_email(spam_dir+spam_folder+'/',emails,'spam'))

In [336]:
spamList = [pd.DataFrame(df,
                      columns=['Year','Month','Day','Weekday',"Hour",
                                'From','To','cc','bcc','Content_type','Subjects','Content','Classification'])for df in spam_results]
spamDF = pd.concat(spamList,ignore_index=True)

In [321]:
pd.options.display.max_colwidth = 1000